Итоговая работа
Веб приложение для прогнозирования параметра влажности
Болотов М.В. АСУ4-22-1м

project_folder/
├── static/
│   └── forecast_plot.png
├── templates/
│   └── index.html
├── Web_Bolotov.py
└── indications.csv

<! - -для шаблона index.html-->
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Прогноз влажности</title>
</head>
<body>
    <h1>Прогноз влажности</h1>
    <img src="{{ plot_url }}" alt="Прогноз влажности">
</body>
</html>

In [1]:
import os                                                                                                       # Импорт необходимых библиотек
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
from flask import Flask, render_template, url_for

In [2]:
app = Flask(__name__)

def forecasting():                                                                                              # Чтение данных из CSV-файла
    indications = pd.read_csv(r"C:\Users\79125\OneDrive\Рабочий стол\Учёба ПНИПУ АСУ\3 семестр\Интеллектуальный анализ Web данных\Итоговая\indications.csv", index_col='Time', parse_dates=True)

    alpha = 0.2                                                                                                 # Параметр для экспоненциального сглаживания

    exp_smoothing = [indications['#Indications'][0]]                                                            # Расчет экспоненциального сглаживания

    for i in range(1, len(indications['#Indications'])):
        exp_smoothing.append(alpha * indications['#Indications'][i] + (1 - alpha) * exp_smoothing[i - 1])

    last_Time = indications.iloc[[-1]].index                                                                    # Генерация времени для следующего показания
    last_Time = last_Time + timedelta(hours=1)

    indications = indications.append(pd.DataFrame(index=last_Time))                                             # Добавление новой строки с предсказанием в данные
    indications['Exp_smoothing'] = exp_smoothing
    indications['Exp_smoothing'] = indications['Exp_smoothing'].shift(1)

    plt.figure(figsize=(15, 8))                                                                                 # Создание графика
    plt.plot(indications['#Indications'], label='Данные с датчика влажности', color='steelblue')
    plt.plot(indications['Exp_smoothing'], label='Экспоненциальное сглаживание', color='orange')

    plt.legend(title='', loc='upper left', fontsize=14)                                                         # Добавление легенды и сетки
    plt.grid(True)

    plt.ylabel('Влажность полотна', fontsize=14)                                                                # Настройка меток и заголовка
    plt.xlabel('Время', fontsize=14)
    plt.title('Показания влажности картонного полотна. Прогноз на следующее показание', fontsize=16)

    plot_path = "static/forecast_plot.png"                                                                      # Сохранение графика в файл
    plt.savefig(plot_path)
    
    return plot_path                                                                                            # Возвращаем путь к графику

In [3]:
@app.route('/')
def home():
    plot_path = forecasting()                                                                                   # Получение пути к графику
    return render_template('index.html', plot_url=plot_path)                                                    # Отрисовка HTML-шаблона и передача пути к графику в шаблон

In [4]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)                                                                                  # Запуск приложения в режиме отладки с отключением автоматической перезагрузки

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
